<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

## update user contributions


In [1]:
## should do this on a monthly basis because 1) this is will quickly use up BULK API's allotted quantity
## and 2) db2 only updates some usage numbers on a yearly or monthly basis

## currently this script takes over 25 minutes to upload to SF

In [2]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
#object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",     
}
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
import sys
#sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

ModuleNotFoundError: No module named 'DB2SalesforceAPI'

## Obtain information from DB2 

In [ ]:
#pre-filtering queries using datetime
date_temp = datetime.datetime.today()
year = date_temp.year
month = date_temp.month
period = 14
if month < 10:
    date_str0 = '\''+ str(year)+'-0'+str(month)+'-00'+'\''
    date_str1 = '\''+str(year)+'-0'+str(month)+'-01'+'\''
else:
    date_str0 = '\''+ str(year)+'-'+str(month)+'-00'+'\''
    date_str1 = '\''+str(year)+'-'+str(month)+'-01'+'\''

In [ ]:
print(date_str0)
print(date_str1)

In [ ]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))

In [ ]:
# extract the wiki contributions
sql_query = "select hits, created_by from jos_wiki_pages where state = '1' and scope = 'site';"
wiki_results_df = pd.read_sql_query(sql_query,engine)
#corresponds to wiki_usage__c, wiki_contributions__c

In [ ]:
# extract paper contributions; need investigate how to get the usage per paper
sql_query = "select authorid, hits from jos_resources left join jos_author_assoc "\
            "on jos_author_assoc.subid = jos_resources.id where type = '3' and access in ('0','3','1') "\
            "and standalone = '1' and published in ('1');"
papers_df = pd.read_sql_query(sql_query,engine)

In [ ]:
# extract presentation contributions; need to see how to get usage per presentation
sql_query = "select authorid, hits from jos_resources left join jos_author_assoc "\
            "on jos_author_assoc.subid = jos_resources.id where type = '1' and access in ('0','3','1') "\
            "and standalone = '1' and published in ('1');"
presentation_df = pd.read_sql_query(sql_query,engine)

In [ ]:
# extract presentation contributions; need to see how to get usage per presentation
sql_query = "select authorid, hits from jos_resources left join jos_author_assoc "\
            "on jos_author_assoc.subid = jos_resources.id where type = '7' and access in ('0','3','1') "\
            "and standalone = '1' and published in ('1');"
tools_df = pd.read_sql_query(sql_query,engine)

In [ ]:
# getting the users that used the contributions
sql_query = "select * from jos_author_stats WHERE datetime " \
    "between " + date_str0 +" and " + date_str1 + " and period = 14;"

result_users_df = pd.read_sql_query(sql_query, engine)

In [ ]:
display(wiki_results_df.head(2))
display(papers_df.head(2))
display(presentation_df.head(2))
display(tools_df.head(2))
display(result_users_df.head(2))
print(result_users_df.shape)

In [ ]:
## get rid of all negative authorids from all of the dataframes
wikidrops = []
wikikeeps = []
for ind,val in enumerate(wiki_results_df['created_by'].to_list()):
    if val <= 0:
        wikidrops.append(ind)
    else:
        wikikeeps.append(ind)
wiki_results_df = wiki_results_df.iloc[wikikeeps,:]

In [ ]:
display(wiki_results_df.head(2))

In [ ]:
## papers df 
pdrops = []
pkeeps = []
for ind,val in enumerate(papers_df['authorid'].to_list()):
    if val <= 0:
        pdrops.append(ind)
    else:
        pkeeps.append(ind)
papers_df = papers_df.iloc[pkeeps,:]

In [ ]:
display(papers_df.head(2))

In [ ]:
## presentation_df
prdrops = []
prkeeps = []
for ind,val in enumerate(presentation_df['authorid'].to_list()):
    if val <= 0:
        prdrops.append(ind)
    else:
        prkeeps.append(ind)
presentation_df = presentation_df.iloc[prkeeps,:]

In [ ]:
display(presentation_df.head(2))

## roll up information from wiki_results_df, papers_df, and presentation_df

In [ ]:
# wiki_results_df

## get the number of unique wiki contributions
wikic = wiki_results_df.groupby(['created_by']).size().reset_index() #need reset_index to save the authorid/created_by field
wikic.columns = ['nanoHUB_user_ID__c','wiki_contributions__c']
wikic = pd.DataFrame(wikic)

## rollup usage
wikic2 = wiki_results_df.groupby(['created_by']).sum().reset_index()
wikic2.columns = ['nanoHUB_user_ID__c','wiki_usage__c']
wikic2 = pd.DataFrame(wikic2)

wikic = wikic.join(wikic2,lsuffix='1', rsuffix='2')
display(wikic.head(2))

In [ ]:
wikic = wikic.drop(columns=['nanoHUB_user_ID__c2'])
wikic = wikic.rename(columns={"nanoHUB_user_ID__c1": "nanoHUB_user_ID__c"})
display(wikic.head(2))

In [ ]:
# papers_df - just need the number of unique contributions

## get the number of unique paper contributions
papersc = papers_df.groupby(['authorid']).size().reset_index() #need reset_index to save the authorid/created_by field
papersc.columns = ['nanoHUB_user_ID__c','paper_contributions__c']
papersc = pd.DataFrame(papersc)

## to add in the number of paper hits in the future ##TODO

display(papersc.head(2))

In [ ]:
# presentation_df - just need the number of unique contributions

## get the number of unique presentation contributions
pres_c = presentation_df.groupby(['authorid']).size().reset_index() #need reset_index to save the authorid/created_by field
pres_c.columns = ['nanoHUB_user_ID__c','presentations_contributions__c']
pres_c = pd.DataFrame(pres_c)

## to add in the number of paper hits in the future ##TODO

display(pres_c.head(2))

## Big picture users dataframe development

In [ ]:
# rename the authorid in result_users_df, then join all of the dataframes
ovr_df = result_users_df.rename(columns={"authorid": "nanoHUB_user_ID__c"})
ovr_df['contributor__c'] = 0

In [ ]:
## get rid of all negative nanohub ids
drops = []
keeps = []
for ind,val in enumerate(ovr_df['nanoHUB_user_ID__c'].to_list()):
    if val <= 0:
        drops.append(ind)
    else:
        keeps.append(ind)
ovr_df = ovr_df.iloc[keeps,:]

ovr_df = ovr_df.drop(columns=['datetime','period','id'])

display(ovr_df.head(2))
display(ovr_df.tail(2))

In [ ]:
ovr_df = ovr_df.set_index('nanoHUB_user_ID__c').join(wikic.set_index('nanoHUB_user_ID__c')) 
ovr_df = ovr_df.join(papersc.set_index('nanoHUB_user_ID__c')) 
ovr_df = ovr_df.join(pres_c.set_index('nanoHUB_user_ID__c')) 

display(ovr_df.head(10))
#use .reset_index() to extract it later

## Get the current nanohub user IDs and their tool counts from SF
then begin the dataframe merge and post-processing

In [ ]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

In [ ]:
# get nanohub ID for contacts to being matching and processing
sf_username_df = db_s.query_data('SELECT nanoHUB_user_ID__c,Total_tool_contributions__c FROM Contact')

# display
display(sf_username_df.head(2))
print(sf_username_df.shape)

In [ ]:
# join the two dataframes
nover_df = sf_username_df.set_index('nanoHUB_user_ID__c').join(ovr_df.set_index('nanoHUB_user_ID__c'))

In [ ]:
display(nover_df.head(2))

In [ ]:
## calculate the non tool contributions
nover_df['And_More_Contributions__c'] = nover_df['wiki_contributions__c']+nover_df['paper_contributions__c']\
        +nover_df['presentations_contributions__c']

In [ ]:
# recover nanohub ids 
nover_df = nover_df.reset_index()
display(nover_df.head(2))

In [ ]:
## remove NAN rows for index
nover_df = nover_df.dropna(subset=['nanoHUB_user_ID__c'])
nover_df.shape

In [ ]:
## then make the indexes integers
import numpy as np
nover_df['nanoHUB_user_ID__c'] = nover_df['nanoHUB_user_ID__c'].apply(lambda x: int(x))
display(nover_df.head(2))

In [ ]:
# calculate contributor flag
for index,val in enumerate(nover_df['contributor__c'].to_list()):
    # if any of the entries are non zero/non-NaN, then contributor__c = 1
    if nover_df['total_users'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['wiki_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['paper_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['presentations_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['Total_tool_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
        
display(nover_df.head(10))

## Fitting data to SF format

In [ ]:
## creating and populating the sf dataframe
df_sf = pd.DataFrame()
temp_df = nover_df.fillna(0)

# populating df_sf and make integers
df_sf['nanoHUB_user_ID__c'] = temp_df['nanoHUB_user_ID__c'].apply(lambda x: int(x))
df_sf['And_More_Contributions__c'] = temp_df['And_More_Contributions__c'].apply(lambda x: int(x))
df_sf['contributed_tool_usage__c'] = temp_df['tool_users'].apply(lambda x: int(x))
df_sf['contributed_andmore_usage__c'] = temp_df['andmore_users'].apply(lambda x: int(x))
df_sf['contributor__c'] = temp_df['contributor__c']

df_sf['wiki_contributions__c'] = temp_df['wiki_contributions__c']
df_sf['wiki_usage__c'] = temp_df['wiki_usage__c']
df_sf['paper_contributions__c'] = temp_df['paper_contributions__c']
df_sf['presentations_contributions__c'] = temp_df['presentations_contributions__c']

display(df_sf.head(2))

## send data to SF

In [ ]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = 'Contact'
db_s.external_id = 'nanoHUB_user_ID__c'

In [ ]:
# send data to Salesforce
db_s.send_data(df_sf)

In [ ]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

In [ ]:
pprint(db_s.check_bulk_failed_results())